In [ ]:
import csv

def read_fbp_to_csv(fbp_file_path, csv_file_path):
    # Lesen der FBP-Datei und Schreiben der Daten in die CSV-Datei
    with open(fbp_file_path, 'r', encoding='ISO-8859-1') as fbp_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        # CSV-Writer-Objekt erstellen
        csv_writer = csv.writer(csv_file)

        # Jede Zeile der FBP-Datei lesen
        for line in fbp_file:
            # Zeilen aufteilen, wobei "|" als Trennzeichen verwendet wird
            fields = line.strip().split("|")
            # Zeile in die CSV-Datei schreiben
            csv_writer.writerow(fields)

# FBP- und CSV-Dateipfade
fbp_file_path = 'Auslandsdaten1.fbp'
csv_file_path = 'Auslandsdaten1.csv'
import chardet

# Die ersten paar Bytes der Datei lesen, um die Kodierung zu ermitteln
rawdata = open(fbp_file_path, "rb").read(10000)  # Die ersten 10.000 Bytes lesen
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)
# Funktion aufrufen
read_fbp_to_csv("Auslandsdaten1.fbp", "Auslandsdaten1.csv")
read_fbp_to_csv("Auslandsdaten2.fbp", "Auslandsdaten2.csv")
import pandas

#read in the csv file skip the first row of the csv file
df = pandas.read_csv("Auslandsdaten1.csv", skiprows=0, encoding=charenc)
#drop the first 2 columns
df = df.drop(df.columns[[0, 1]], axis=1)
# 
#benenne den header so um:
# C1,C2A,C2B,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27
df.columns = ["B1","B2","B3","B4","B5","B6","B7","B8","B9","B10","B11","B12","B13","B14","B15","B16"]
#speicher die datei als Ausland1neu.csv
df.to_csv('Ausland1neu.csv', index=False)
df.info()

df = pandas.read_csv("Auslandsdaten2.csv", skiprows=1, encoding=charenc)
#drop the first 2 columns
df = df.drop(df.columns[[0, 1]], axis=1)


#benenne den header so um:
# C1,C2A,C2B,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27
df.columns = ["C1","C2A","C2B","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13","C14","C15","C16","C17", "C18","C19","C20","C21","C22","C23","C24","C25","C26","C27"]
#speicher die datei als Ausland1neu.csv
df.to_csv('Ausland2neu.csv', index=False)